In [1]:
#default_exp domain.model

In [2]:
#export
from dataclasses import dataclass, field
from typing import (
    Set, Optional, Dict, Any, List, Tuple
)
from functools import partial
from dpcontracts import require, types, ensure

# Domain Model 

> API details.

In [3]:
#hide
from nbdev.showdoc import *

## Tasks

In [4]:
#export
@dataclass(frozen=True, unsafe_hash=True)
class Task:
    """Represents an Airflow Task
    """

    name: str
    task_type: str
    dependencies: Optional[Set[str]] = field(default=None, hash=False)
    parameters: Optional[dict] = field(default=None, hash=False, repr=False)
    config: Optional[dict] = field(default=None, hash=False, repr=False)

    def __post_init__(self):
        assert self.name != ""
        assert self.task_type != ""
    
    def todict(self):
        operator, config_args = list(self.config.get("operator").items()).pop()
        args = {**config_args, **self.parameters}
        return {"name": self.name,
                "dependencies": self.dependencies,
                "type": operator,
                "args": args}

In [5]:
task = Task(name = 'task3',
            task_type = 'ssh',
            dependencies = [],
            parameters = {'task_id': 'sleep', 'bash_command': 'sleep', 'retries': 3},
            config = {
                'operator': {
                    'BashOperator': {'task_id': 'sleep', 
                                     'bash_command': 'sleep',
                                     'retries': 3}},
                    'imports': {'airflow.operators.bash_operator': ['BashOperator']}})

task.todict()

{'name': 'task3',
 'dependencies': [],
 'type': 'BashOperator',
 'args': {'task_id': 'sleep', 'bash_command': 'sleep', 'retries': 3}}

## Workflow Definition

In [6]:
#export
@dataclass
class WorkflowDefinition:
    """Data holder and validation for YAML workflow file
    """

    name: str
    tasks: Dict[str, dict]
    subtasks: Dict[str, Dict[str, dict]]
    default_args: Dict[str, Any]
    dag_args: Dict[str, Any]
        
    
    def __post_init__(self):
        """Validate workflow inputs
        """
        assert len([*self.tasks]) >= 1, \
            "Workflow data must have at least one task"
        assert all(isinstance(task, str) for task in [*self.tasks]) and \
               not any(task == "" for task in [*self.tasks]), \
            "Workflow tasks must have valid string names"
        
    @staticmethod
    def build(
        name: str, 
        tasks: Dict[str, dict],
        subtasks: Dict[str, dict],
        default_args: Dict[str, Any],
        dag_args: Dict[str, Any],
        **kwargs,
    ) -> "WorkflowDefinition":
        """Builds Workflow definition from YAML workflow file

        Parameters
        ----------
        name : str
            workflow name
        tasks : Dict[str, dict]
            workflow tasks used to create primary DAG
        subtasks : Dict[str, tasks]
            worfklow sub-tasks used to create sub DAGs
        default_args : Dict[str, Any]
            default Airflow DAG arguments
        dag_args : Dict[str, Any]
            custom Airflow DAG arguments

        Returns
        -------
        WorkflowDefinition : Dict[str, Dict[str, dict]]
            attributes[name, tasks, subtasks, default_args, dag_args]
        """
    
        return WorkflowDefinition(name, tasks, subtasks, default_args, dag_args)

## Workflow Configuration

In [7]:
#export
@dataclass
class Config:
    """Data holder for YAML Airflow configuration file.
    Holds mapping of user-defined tasks to Airflow Operators
    """

    task_types: Dict[str, dict]

    @staticmethod
    @require(
        "Configuration file must have at least one task-type",
        lambda args: len([*args.task_types]) >= 1,
    )
    @require(
        "Configuration file must have valid task-type names",
        lambda args: all(
            isinstance(task_type, str) for task_type in [*args.task_types]
        )
        and not any(task_type == "" for task_type in [*args.task_types]),
    )
    @types(task_types=dict)
    def build(task_types: Dict[str, dict], **kwargs) -> "Config":
        return Config(task_types=task_types)

    @types(task_type=str)
    @require(
        "User specified task type must be in configuration file",
        lambda args: args.task_type in [*args.self.task_types],
    )
    def fetch(self, task_type: str) -> "Config":
        """Fetch specific task configuration from configuration file.

        Config.task_types reduced to single task_type requested.
        """
        task_dict = {task_type: self.task_types.get(task_type)}
        return Config(task_types=task_dict)

## Workflow

In [8]:
#export
@dataclass
class Workflow:
    """Defines how to construct a Workflow object from Workflow Definition"""

    name: str
    config: Dict[str, dict]
    tasks: Optional[List[Task]] = None

    def build(self, tasks) -> "Workflow":
        # Extract Tasks from workflow definition
        compose = partial(self.compose_task, self.config)
        tasks = [
            compose((name, definition))
            for name, definition in tasks.items()
        ]
        return Workflow(self.name, self.config, tasks)
    

    @ensure("Task type not found", lambda args, result: result.config is not None)
    def compose_task(self, config: Dict[str, dict], task: Tuple[str, dict]) -> Task:
        # Build Action objects with type specific settings from config file
        name, definition = task
        task_type = definition.get("type")
        return Task(name, 
                    task_type,
                    definition.get("dependencies"),
                    definition.get("parameters"),
                    config.get(task_type))
    
    #TODO
    def imports(self):
        """Extract a unique set of module imports required by the Workflow Definition.
        """
        all_imports = [task.config.get("imports") for task in self.tasks]
        print(all_imports)

In [9]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 02_graph.ipynb.
Converted 03_templates.ipynb.
Converted 04_adapters.ipynb.
Converted index.ipynb.
Converted software_architecture.ipynb.
